## Building flask api

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, '../../')

In [3]:
from src.serve import *

Failed to load FluidSynth. Must install if you want to convert to wav files.


In [4]:
np.set_printoptions(edgeitems=10, threshold=40, linewidth=200)

In [5]:
data_dir = 'data/midi/v9/'
source_dir = 'midi_encode/np/shortdur'

file_path = Path('../../')
path = file_path/data_dir/source_dir
out_path = file_path/'data/generated/'

In [6]:
encoded_path = file_path/'data/assets/cmajor/seed'

## Learn config

In [7]:
import hashlib
import shutil

In [8]:
def df2records(path):
    df = pd.read_csv(path/'midi_encode.csv')
    df = df.loc[df[source_dir].notna()] # make sure it exists
    df = df.loc[df.source == 'hooktheory'] # hooktheory only
    df = df.rename(index=str, columns={source_dir: 'numpy'}) # shortdur -> numpy
    df = df.reindex(index=df.index[::-1]) # A's first
    df = df.where((pd.notnull(df)), None) # nan values break json
    return df.to_dict('records')

In [12]:
def format_meta(s):
    title = s['title'].title().replace('-', ' ')
    artist = s['artist'].title().replace('-', ' ')
    display = ' - '.join([title, artist])
    if s.get('section'): display += ' - ' + s['section'].title()
    sid = hashlib.md5(display.encode('utf-8')).hexdigest()
    
    json_data = {
        'title': title,
        'artist': artist,
        'bpm': s['ht_bpm'],
        'display': display,
        'genres': s['genres'],
        'sid': sid
    }
    source_file = file_path/data_dir/s['midi']
#     key_file = str(source_file).replace('cmajor', 'original')
    key_file = source_file
    to_file = encoded_path/f'{sid[::-1]}.mid'
    to_json = encoded_path/f'{sid[::-1]}.json'
    if not to_file.exists():
        shutil.copy(str(key_file), str(to_file))
        with open(to_json, 'w') as jp:
            json.dump(json_data, jp)
    
    return json_data

In [13]:
def build_db(path):
    recordlist = df2records(path)
    htlist = [format_meta(s) for s in recordlist]
    json_path = file_path/'data/assets/cmajor/json/htlist.json'
    with open(json_path, 'w') as fp:
        json.dump(htlist, fp, separators=(',', ':'))
    return htlist

In [14]:
htlist = build_db(path)